In [2]:
!pip install -qU langchain sentence_transformers
!pip install -qU langchain-huggingface langchain-community
!pip install -qU faiss-cpu
!pip install --upgrade openai
!pip install -qU tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7

In [ ]:
import faiss
import openai
import numpy as np
from google.colab import drive, files
import pickle
import os

# Monta o Google Drive
drive.mount('/content/drive', force_remount=True)

os.environ["OPENAI_API_KEY"] = ""

client = openai.OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Mounted at /content/drive


In [17]:
import pandas as pd

df = pd.read_json("/content/drive/Shareddrives/RAG Receita/extracted_questions_with_embedded_references_20250220_question_extraction_fix.json")
df.head()

,question_number,question_summary,question_text,answer,answer_cleaned,references,linked_questions,formatted_references,embedded_references,formatted_embedded_references
0,1,OBRIGATORIEDADE,Quem está obrigado a apresentar a Declaração d...,[Está obrigada a apresentar a Declaração de Aj...,[Está obrigada a apresentar a Declaração de Aj...,"[Lei nº 9.250, de 26 de dezembro de 1995, art....",[],"[{'título': 'Lei nº 9.250, de 26 de dezembro d...","[Lei nº 11.196, de 21 de novembro de 2005, art...","[{'título': 'Lei nº 11.196, de 21 de novembro ..."
1,2,PESSOA FÍSICA DESOBRIGADA,Pessoa física desobrigada pode apresentar a De...,"[Sim. A pessoa física, ainda que desobrigada, ...","[Sim. A pessoa física, ainda que desobrigada, ...","[Instrução Normativa RFB nº 2.178, de 5 de mar...","[001, 174]","[{'título': 'Instrução Normativa RFB nº 2.178,...",[],None
2,3,TITULAR OU SÓCIO DE EMPRESA,Contribuinte que é titular ou sócio de empresa...,"[Não, a menos que se enquadre nas hipóteses pr...","[Não, a menos que se enquadre nas hipóteses pr...","[Instrução Normativa RFB nº 2.178, de 5 de mar...",[001],"[{'título': 'Instrução Normativa RFB nº 2.178,...",[],None
3,4,QUADRO SOCIETÁRIO OU ASSOCIADO DE COOPERATIVA,"Contribuinte, que participou de quadro societá...","[Não, a menos que esteja obrigado a declarar c...","[Não, a menos que esteja obrigado a declarar c...","[Instrução Normativa RFB nº 2.178, de 5 de mar...",[001],"[{'título': 'Instrução Normativa RFB nº 2.178,...",[],None
4,5,DE DECLARAR,Contribuinte que constou como responsável pera...,[Esse contribuinte está obrigado a declarar ca...,[Esse contribuinte está obrigado a declarar ca...,"[Instrução Normativa RFB nº 2.178, de 5 de mar...",[001],"[{'título': 'Instrução Normativa RFB nº 2.178,...",[],None


In [ ]:
faiss_type = "recursive"

# Caminho para o índice FAISS salvo no Google Drive
folder_path = '/content/drive/Shareddrives/RAG Receita/FAISS/'

if faiss_type == "sliding_window":
    INDEX_PATH = folder_path + "faiss_index_normas_sliding_window.index"
elif faiss_type == "hierarchical":
    INDEX_PATH = folder_path + "faiss_index_normas_hierarchical.index"
else:
    INDEX_PATH = folder_path + "faiss_index_normas_recursive_splitter.index"


# Carrega o índice FAISS
index = faiss.read_index(INDEX_PATH)

openai.api_key = ""
CHUNKS_SW_PATH = "/content/drive/Shareddrives/RAG Receita/Chunks/chunks_normas_sliding_window.pkl"
CHUNKS_REC_PATH = "/content/drive/Shareddrives/RAG Receita/Chunks/chunks_normas_recursive.pkl"
CHUNKS_HIER_PATH = "/content/drive/Shareddrives/RAG Receita/Chunks/chunks_normas_hierarchical.pkl"

# Carrega as listas de chunks
def load_chunks(path):
    with open(path, "rb") as f:
        return pickle.load(f)

chunks_sliding_window = load_chunks(CHUNKS_SW_PATH)
chunks_recursive = load_chunks(CHUNKS_REC_PATH)
chunks_hierarchical = load_chunks(CHUNKS_HIER_PATH)

# Função para obter embeddings da OpenAI
def get_embedding(text):
    response = openai.embeddings.create(
        model="text-embedding-3-small",  # OpenAI Small Embedding
        input=text
    )
    return np.array(response.data[0].embedding, dtype=np.float32)

# Função de busca no índice FAISS
def search_faiss(index, query, chunk_type="recursive", top_k=5):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)

    # Escolhe a lista de chunks correspondente
    if chunk_type == "sliding_window":
        chunks = chunks_sliding_window
    elif chunk_type == "hierarchical":
        chunks = chunks_hierarchical
    else:
        chunks = chunks_recursive

    # Recupera os textos correspondentes aos índices
    resultados = [chunks[i] for i in indices[0] if i < len(chunks)]

    return resultados

# Teste
query = "exemplo de consulta"
resultados = search_faiss(index, query, chunk_type=faiss_type)
print(resultados)

['Solução de Consulta Cosit nº 70: NORMAS Contraste \ue88a \ue8ad Solução de Consulta Cosit nº 70, de 03 de abril de 2024 (Publicado(', 'Solução de Consulta Cosit nº 64: NORMAS Contraste \ue88a \ue8ad Solução de Consulta Cosit nº 64, de 26 de março de 2024 (Publicado(', 'Solução de Consulta Cosit nº 214: NORMAS Contraste \ue88a \ue8ad Solução de Consulta Cosit nº 214, de 19 de julho de 2024 (Publicado(', "Solução de Consulta Cosit nº 7: § 1 , II, 'c', e IV, 'c', e 215, caput e", 'Solução de Divergência Cosit nº 19, de 16 de setembro de 2013: a) no DOU de 07/10/2013, seção 1, página 16) Multivigente Vigente Original Relacional ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA FÍSICA - IRPF EMENTA: GANHO DE CAPITAL. MEAÇÃO. HERANÇA. A meação não surge com a morte de um dos cônjuges, ela existe enquanto durar o casamento. Na transferência do direito por herança, há transferência do patrimônio do de cujus para o herdeiro. Já na meação, não há transferência, pois a parcela do cônjuge meei

In [18]:
def get_rag_response(pergunta, index, faiss_type, top_k):

  contextos = search_faiss(index, pergunta, chunk_type=faiss_type, top_k=top_k)
  contexto = " ".join(contextos)

  prompt = f'''
    Você é um assistente virtual especializado em tributos brasileiros da Receita Federal. Responda à pergunta de forma completa e com base no contexto fornecido. Para garantir uma resposta precisa e fundamentada, siga estas etapas:

    Análise da Pergunta: Identifique o que a pergunta busca, considerando palavras-chave e aspectos específicos.

    Busca no Contexto: Examine o contexto em busca de trechos relevantes que se relacionem diretamente com a pergunta.

    Construção da Resposta:

    - Utilize as informações do contexto e organize a resposta de forma objetiva e precisa.
    - Cite diretamente as normas ou leis, utilizando as referências exatas sempre que possível, sem mencionar o termo "contexto", "referências", "fontes".
    - Avaliação de Suficiência: Se o contexto não for suficiente para responder de forma fundamentada, responda apenas: "Estou aprendendo e no futuro poderei responder a essa pergunta."

    Regras a seguir:

    - Não inclua frases de interação direta com o usuário.
    - Caso o contexto não seja suficiente, declare que a resposta será disponibilizada futuramente.
    - Se a resposta já tiver sido respondida, não procure mais informações para complementar em outros contextos.
    - Sempre cite a norma e artigo que foi retirada a informação ao final da resposta em formato de lista, para o usuário consultar, ela geralmente esta no começo dos contextos (serão enviados 3 em uma única string) antes dos ':' por exemplo: Lei nº 9.250, de 26 de dezembro de 1995, art. 25 ou Instrução Normativa RFB nº 2.178, de 5 de março de 2024, art. 2º.
    - Jamais cite parágrafos, incisos, §, alíneas e numerações, apenas normas e artigos nas respostas nem na lista.
    - Não mencione no meio da respostas nenhuma norma, artigo, parágrafos, incisos (§), alíneas e numerações, apenas na lista final de consulta do usuário.
    - Substitua RIR/2018 ou RIR2018 quando for citar ele, o nome real que deve ser citado é Decreto Nº 9.580.
    - Para o caso de acórdãos, a informação de qual é fica no contexto, não no nome e geralmente é assim: "Acórdão nº: 303-25.277 Nº do processo: 11065-001.472/87-94 Contribuinte: Bolivar & CIA", caso use algum em alguma resposta, cite-o na lista final também.
    - Não fale que consultou a normas ou artigos, apenas forneça a lista.

  Pergunta a ser respondida: "{pergunta}"

  Contexto: "{contexto}"'''

  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-4o-mini")

  return chat_completion.choices[0].message.content, contextos

In [19]:
df[['resposta_rag', 'contexto']] = df["question_text"].apply(lambda pergunta: pd.Series(get_rag_response(pergunta, index, faiss_type, 5)))

In [20]:
df.to_excel("teste_total.xlsx", index = False)

In [23]:
files.download('teste_total.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>